In [20]:
import numpy as np
import matplotlib.pyplot as plt
from Regler_class_file import PI_controller_class

#importing Druckrohrleitung
import sys
import os
current = os.path.dirname(os.path.realpath('Main_Programm.ipynb'))
parent = os.path.dirname(current)
sys.path.append(parent)
from functions.pressure_conversion import pressure_conversion
from Ausgleichsbecken.Ausgleichsbecken_class_file import Ausgleichsbecken_class
from Turbinen.Turbinen_class_file import Francis_Turbine

In [21]:
#define constants

#Turbine
Q_nenn          = 0.85  # m³/s
p_nenn          = pressure_conversion(10.6,'bar','Pa')
closing_time    = 480.   #s

# physics
g               = 9.81                          # gravitational acceleration [m/s²]
rho             = 1000.                         # density of water [kg/m³]

# define controller constants
target_level = 8.  # m
Kp = 0.01
Ti = 200.
deadband_range = 0.05 # m

# reservoir
initial_level               = target_level
initial_influx              = Q_nenn/2        # initial influx  of volume to the reservoir [m³/s]
initial_pressure_unit       = 'Pa'              # DO NOT CHANGE! for pressure conversion in print statements and plot labels 
conversion_pressure_unit    = 'bar'             # for pressure conversion in print statements and plot labels
area_base                   = 74.               # total base are of the cuboid reservoir [m²]   
area_outflux                = 1.                 # outflux area of the reservoir, given by pipeline area [m²]
critical_level_low          = 0.                # for yet-to-be-implemented warnings[m]
critical_level_high         = np.inf            # for yet-to-be-implemented warnings[m]

p0 = rho*g*initial_level-0.5*rho*(initial_influx/area_outflux)**2

# offset the pressure in front of the turbine to get realisitc fluxes
h_fict = 100
offset_pressure = rho*g*h_fict

t_max                       = 1e4 #s
dt                          = 1e-2      # simulation timestep
nt                          = int(t_max//dt)    # number of simulation steps of reservoir in between timesteps of pipeline              

t_vec = np.arange(0,nt+1,1)*dt



In [22]:
# create objects

V = Ausgleichsbecken_class(area_base,area_outflux,critical_level_low,critical_level_high,dt)
V.set_steady_state(initial_influx,initial_level,conversion_pressure_unit)

T1 = Francis_Turbine(Q_nenn,p_nenn,closing_time,dt)
T1.set_steady_state(initial_influx,p0+offset_pressure)

Pegelregler = PI_controller_class(target_level,deadband_range,Kp,Ti,dt)

In [23]:
level_vec       = np.full(nt+1,V.level)
LA_ist_vec      = np.full(nt+1,T1.LA)
LA_soll_vec     = np.full(nt+1,T1.LA)
Q_vec           = np.full(nt+1,initial_influx)

Pegelregler.control_variable = T1.get_current_LA()

In [24]:
# time loop

for i in range(nt+1):

    if np.mod(i,1e4) == 0:
        print(t_vec[i])

    if i == 0.2*(nt+1):
        V.set_influx(0.)
    elif i == 0.5*(nt+1):
        V.set_influx(0.5*Q_nenn)
    elif i == 0.8*(nt+1):
        V.set_influx(Q_nenn)


    p = V.get_current_pressure()
    Pegelregler.update_control_variable(V.level)
    LA_soll = Pegelregler.get_current_control_variable()
    T1.update_LA(LA_soll)
    T1.set_pressure(p+offset_pressure)
    LA_soll_vec[i] = LA_soll
    LA_ist_vec[i]  = T1.get_current_LA()
    Q_vec[i] = T1.get_current_Q()

    
    V.set_outflux(Q_vec[i])

    V.timestep_reservoir_evolution()                                                              
                                                                                                                   
    level_vec[i] = V.get_current_level()
    
    

0.0
100.0
200.0
300.0
400.0
500.0
600.0
700.0
800.0
900.0
1000.0
1100.0
1200.0
1300.0
1400.0
1500.0
1600.0
1700.0
1800.0
1900.0
2000.0
2100.0
2200.0
2300.0
2400.0
2500.0
2600.0
2700.0
2800.0
2900.0
3000.0
3100.0
3200.0
3300.0
3400.0
3500.0
3600.0
3700.0
3800.0
3900.0
4000.0
4100.0
4200.0
4300.0
4400.0
4500.0
4600.0
4700.0
4800.0
4900.0
5000.0
5100.0
5200.0
5300.0
5400.0
5500.0
5600.0
5700.0
5800.0
5900.0
6000.0
6100.0
6200.0
6300.0
6400.0
6500.0
6600.0
6700.0
6800.0
6900.0
7000.0
7100.0
7200.0
7300.0
7400.0
7500.0
7600.0
7700.0
7800.0
7900.0
8000.0
8100.0
8200.0
8300.0
8400.0
8500.0
8600.0
8700.0
8800.0
8900.0
9000.0
9100.0
9200.0
9300.0
9400.0
9500.0
9600.0
9700.0
9800.0
9900.0


In [25]:
%matplotlib qt5
# time loop

# create a figure and subplots to display the velocity and pressure distribution across the pipeline in each pipeline step
fig1,axs1 = plt.subplots(3,1)
axs1[0].set_title('Level')
axs1[0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs1[0].set_ylabel(r'$h$ [$\mathrm{m}$]')
axs1[0].plot(t_vec,level_vec)
axs1[0].set_ylim([0*initial_level,1.5*initial_level])
axs1[1].set_title('Flux')
axs1[1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs1[1].set_ylabel(r'$Q$ [$\mathrm{m} / \mathrm{s}^3$]')
axs1[1].plot(t_vec,Q_vec)
axs1[1].set_ylim([0,2*initial_influx])
axs1[2].set_title('LA')
axs1[2].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs1[2].set_ylabel(r'$LA$ [%]')
axs1[2].plot(t_vec,LA_soll_vec)
axs1[2].plot(t_vec,LA_ist_vec)
axs1[2].set_ylim([0,1])
fig1.tight_layout()
fig1.show()


In [26]:
fig2 = plt.figure()
plt.plot(t_vec,Pegelregler.get_error_history())